In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the Data
# Drop the columns that are not required

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
## Encode the categorical variables to numerical variables

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [5]:
# One Hot Encoding the Geography column


onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')                   ## handle_unknown='ignore' will ignore the unknown categories.
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
## Combine one-hot encoded columns with the original data

data = pd.concat([data.drop('Geography', axis =1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## Split the data into features and target

X = data.drop('EstimatedSalary', axis = 1)   ## Droping the target column from the features
Y = data['EstimatedSalary']                  ## Target column

In [9]:
## Split the data into training and testing data
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [11]:
## Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)    # Fit and transform the training data 
X_test =  scaler.transform(X_test)         # Transform the testing data


In [12]:
## Save the encoder and scalers for later use

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Regression Problem statement

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
## Create the model
model = Sequential([
    Dense(64, activation = 'relu', input_shape=(X_train.shape[1],)),   #input_shape is the number of features where X_train.shape[1] gives the no of features
    Dense(32, activation = 'relu'),
    Dense(1)

])

In [16]:
## Compile the model

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

#Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  ## Create a log directory with the current timestamp
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)                ## Create a tensorboard callback

In [18]:
## Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [19]:
## Train the model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 100353.0781 - mae: 100353.0781 - val_loss: 98434.2266 - val_mae: 98434.2266
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99367.2031 - mae: 99367.2031 - val_loss: 96497.7812 - val_mae: 96497.7812
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96036.4688 - mae: 96036.4688 - val_loss: 91686.1016 - val_mae: 91686.1016
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 89665.5312 - mae: 89665.5312 - val_loss: 83933.0156 - val_mae: 83933.0156
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 80715.2109 - mae: 80715.2109 - val_loss: 74382.1328 - val_mae: 74382.1328
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 70825.3047 - mae: 70825.3047 - val_loss: 65160.6484 - val_mae: 65160.6484
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 62003.2578 - mae: 62003.257

In [20]:
## Evaluate the model
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 27092), started 0:01:02 ago. (Use '!kill 27092' to kill it.)

In [25]:
## Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, Y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 1ms/step - loss: 50303.4492 - mae: 50303.4492
Test MAE:50303.44921875


In [26]:
## Save the Model
model.save('regression_model.h5')

b:\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
